# Analisi univariata e bivariata


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder



In [ ]:
df=pd.read_csv('../docs/csv/csvscenarioTest1v1.csv',index_col=0)

In [ ]:
c=df.select_dtypes(include='object').columns.tolist()


In [ ]:
le_df=df
le = preprocessing.LabelEncoder()
le_df[c] = df[c].apply(lambda col: le.fit_transform(col))

In [ ]:
x=le_df.columns
y=le_df['win']

In [ ]:
for i in x:
    sns.FacetGrid(le_df,hue='win',size=5).map(sns.distplot,i).add_legend()

In [ ]:
for i in range(len(x)):
    plt.figure()
    plt.plot(df[x[i]],y)
    # Show/save figure as desired.
    plt.show()

In [ ]:
x=le_df.columns
y=le_df['win']
for i in range(len(x)):
    plt.scatter(df.index,df[x[i]])
    plt.show()

In [ ]:
x=le_df.columns
y=le_df['win']
for i in range(len(x)):
    plt.hist(le_df[x[i]])
    plt.show()

In [ ]:
!pip install --user -U pandas-profiling

In [ ]:
df['stat-name_red_Tank1'].value_counts().plot.bar()


In [ ]:
df['hp_red_Tank1'].value_counts().sort_index().plot.line()

In [ ]:

df['hp_red_Tank1'].plot.hist()

In [ ]:
le_df.plot.scatter(x='move_red_Tank1', y='win')

In [ ]:
for i in range(len(x)):
    sns.boxplot(x=x[i],y='win',data=le_df)
    plt.show()